In [ ]:
import math
import pandas as pd
import altair as alt
import numpy as np
import json

In [ ]:
filename = 'docs/assets/metadata.json'
f = open(filename, 'r')
j = json.load(f)
df = pd.DataFrame({'speaker':[], 'param':[], 'value':[]})
count = 0
for i in j:
    speaker = j[i]
    for p in speaker:
        val = speaker[p]
        if type(val) is dict:
            for v in val.keys():
                #print('{0} {1} {2}'.format(i, v, val[v]))
                df.loc[count] = [i, v, val[v]]
                count += 1
        else:
            # print('{0} {1} {2}'.format(i, p, val))
            df.loc[count] = [i, p, val]
            count += 1
df.head()

In [ ]:
pref_score = df.loc[(df.param=='pref_score')].reset_index()
brand = df.loc[(df.param=='brand')].reset_index()
lfx_hz = df.loc[(df.param=='lfx_hz')].reset_index()
nbd_on = df.loc[(df.param=='nbd_on_axis')].reset_index()
nbd_pir = df.loc[(df.param=='nbd_pred_in_room')].reset_index()
sm_pir = df.loc[(df.param=='sm_pred_in_room')].reset_index()

source = pd.DataFrame({
    'speaker': pref_score.speaker, 
    'brand': brand.value,
    'pref_score': pref_score.value, 
    'lfx_hz': lfx_hz.value,
    'nbd_on': nbd_on.value,
    'nbd_pir': nbd_pir.value,
    'sm_pir': sm_pir.value,
}).dropna()

In [ ]:
lfx_hz = alt.Chart(source).mark_point(
).encode(
    x=alt.X('lfx_hz:Q'),
    y=alt.Y('pref_score:Q'),
    color=alt.Color('brand:N'),
    tooltip=['speaker', 'lfx_hz', 'pref_score']
)

g_lfx = lfx_hz+lfx_hz.transform_regression('lfx_hz', 'pref_score').mark_line()

In [ ]:
nbd_on = alt.Chart(source).mark_point(
).encode(
    x=alt.X('nbd_on:Q'),
    y=alt.Y('pref_score:Q'),
    color=alt.Color('brand:N'),
    tooltip=['speaker', 'nbd_on', 'pref_score']
)

g_nbd_on = nbd_on+nbd_on.transform_regression('nbd_on', 'pref_score').mark_line()

In [ ]:
nbd_pir = alt.Chart(source).mark_point(
).encode(
    x=alt.X('nbd_pir:Q'),
    y=alt.Y('pref_score:Q'),
    color=alt.Color('brand:N'),
    tooltip=['speaker', 'nbd_pir', 'pref_score']
)

g_nbd_pir = nbd_pir+nbd_pir.transform_regression('nbd_pir', 'pref_score').mark_line()

In [ ]:
sm_pir = alt.Chart(source).mark_point(
).encode(
    x=alt.X('sm_pir:Q'),
    y=alt.Y('pref_score:Q'),
    color=alt.Color('brand:N'),
    tooltip=['speaker', 'sm_pir', 'pref_score']
)

g_sm_pir = sm_pir+sm_pir.transform_regression('sm_pir', 'pref_score').mark_line()

In [ ]:
(g_lfx|g_nbd_on)&(g_nbd_pir|g_sm_pir)

In [ ]:
distribution = alt.Chart(source).mark_bar().encode(
    x=alt.X('pref_score:Q', bin=True),
    y='count()'
).properties(width=300,height=300)

In [ ]:
spread = alt.Chart(source).mark_circle(size=30).encode(
    x=alt.X('speaker', sort='y', axis=alt.Axis(labelAngle=45)),
    y=alt.Y('pref_score')
).properties(width=700,height=300)

In [ ]:
spread|distribution